In [1]:
import urllib3
import os

def open_matrix(filename='outfile_kuba.txt'):
    with open(filename, 'r') as f:
        l = [[int(float(num)) for num in line.split(' ')] for line in f]
    return l


def get_coco_labels():
    http = urllib3.PoolManager()
    r = http.request('GET', 'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt')
    dane = r.data.decode('UTF-8')
    my_data = dane.split('\n')

    coco = {i+1: my_data[i] for i in range(0,len(my_data))}
    coco.pop(91) # remove hairbrush
    return coco

In [ ]:
coco = get_coco_labels()

coco_readable_to_index = {value:key for key, value in coco.items()}

def get_all_detections(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][:-1]

def get_TP(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][coco_readable_to_index[class_name]-1]

# not detected
def get_FN(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][-1]

def get_FP(confusion_matrix, class_name):
    return sum(get_all_detections(confusion_matrix, class_name)) - get_TP(class_name)



def get_precision(confusion_matrix, class_name):
    return get_TP(confusion_matrix, class_name) / get_all_detections(confusion_matrix, class_name)

def get_recall(confusion_matrix, class_name):
    return get_TP(confusion_matrix, class_name) / ((get_TP(confusion_matrix, class_name))+get_FN(confusion_matrix, class_name))

def get_F1_score(confusion_matrix, class_name):
    numerator = 2 * get_TP(confusion_matrix, class_name)
    denominator = numerator + get_FP(confusion_matrix, class_name) + get_FN(confusion_matrix, class_name)

    return numerator/denominator

In [20]:
# tmp = []
# for row in confusion_matrix:
#     normalized_row = []
#     for value in row:
#         normalized_row.append(value/(sum(row)))
#     tmp.append(normalized_row)


def read_matrices(model_name):
    matrices = []
    files = os.listdir(model_name)
    for file in files:
        if file.endswith(".txt"):
            resolution = file.split("_")[0]
            if resolution.split("x")[0] != "None":
                resolution_int = int(resolution.split("x")[0]) * int(resolution.split("x")[1])
            else:
                resolution_int = 0
            matrices.append((resolution_int, resolution, open_matrix(model_name + "\\" + file)))

    matrices.sort(key = lambda x: x[0])
    return matrices

In [21]:

matrices = read_matrices("ssd+mobilenet_v2")